# DALI Binary Arithmetic Operators - Type Promotions

This example describes the rules regarding the type promotions for binary arithmetic operators in DALI. See "DALI expressions and arithmetic operators" for more information about using arithmetic operators in DALI.

## Prepare the Test Pipeline

1. Prepare the helper code, so we can easily manipulate the types and values that will appear as tensors in the DALI pipeline.

2. Numpy will be used as the source for the custom provided data, so we need to import several things from DALI, to create the pipeline and use the `ExternalSource` operator. 

In [1]:
import numpy as np
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.types import Constant

batch_size = 1

### Defining the Pipeline

To define the data, because there are binary operators, we need two inputs.

1. Create a simple helper function that returns two numpy arrays of given numpy types with arbitrary selected values.

   This simplifies the manipulation of types. In an actual scenario the data that is processed by the DALI arithmetic operators will be tensors that are produced by another operator that contains some          images, video sequences, or other data.

**Note**: The shapes of both inputs need to match since operations are performed element-wise.  

In [2]:
left_magic_values = [42, 8]
right_magic_values = [9, 2]


def get_data(left_type, right_type):
    return ([left_type(left_magic_values)], [right_type(right_magic_values)])


batch_size = 1

2. To define the pipeline, the data will be obtained from the get_data function and made available to the pipeline through ExternalSource.

**Note**: You do not need to instantiate any additional operators, we can use regular Python arithmetic expressions on the results of other operators.

3. For convenience, wrap the arithmetic operations usage in a lambda called `operation`, which is specified when creating the pipeline.

In [3]:
def dali_type(np_type):
    return types.to_dali_type(np.dtype(np_type).name)


def arithmetic_pipeline(operation, left_type, right_type):
    pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
    with pipe:
        l, r = fn.external_source(
            source=lambda: get_data(left_type, right_type),
            num_outputs=2,
            dtype=[dali_type(left_type), dali_type(right_type)],
        )
        pipe.set_outputs(l, r, operation(l, r))

    return pipe

## Type Promotion Rules

Type promotions for binary operators are described below. The type promotion rules are commutative, and they apply to `+`, `-`, `*`, and `//`. The `/` always returns a float32 for integer inputs, and applies the rules below when at least one of the inputs is a floating point number.

| Operand Type | Operand Type | Result Type | Additional Conditions |
|:------------:|:------------:|:-----------:| --------------------- |
| T      | T      | T                |                        |
| floatX | T      | floatX           | where T is not a float |
| floatX | floatY | float(max(X, Y)) |                        |
| intX   | intY   | int(max(X, Y))   |                        |
| uintX  | uintY  | uint(max(X, Y))  |                        |
| intX   | uintY  | int2Y            | if X <= Y              |
| intX   | uintY  | intX             | if X > Y               |

The `bool` type is the smallest unsigned integer type and is treated as `uint1` with respect to the table above.

The bitwise binary  `|`, `&`, and `^` operations abide by the same type promotion rules as arithmetic binary operations, but their inputs are restricted to integral types (bool included).

Only multiplication `*` and bitwise operations `|`, `&`, `^` can accept two `bool` inputs.

### Using the Pipeline

1. Create a pipeline that adds two tensors of type `uint8`, run it, and check the results:

In [4]:
def build_and_run(pipe, op_name):
    pipe.build()
    pipe_out = pipe.run()
    l = pipe_out[0].as_array()
    r = pipe_out[1].as_array()
    out = pipe_out[2].as_array()
    print(
        "{} {} {} = {}; \n\twith types {} {} {} -> {}\n".format(
            l, op_name, r, out, l.dtype, op_name, r.dtype, out.dtype
        )
    )


pipe = arithmetic_pipeline((lambda x, y: x + y), np.uint8, np.uint8)
build_and_run(pipe, "+")

[[42  8]] + [[9 2]] = [[51 10]]; 
	with types uint8 + uint8 -> uint8



Here is some information about how all of the operators behave with different type combinations by generalizing the example above.
You can use the `np_types` or `np_int_types` in the loops to see all possible type combinations. Only a few combinations have been used. You can also set some additional printing options for numpy to make the output more aligned.

In [5]:
np.set_printoptions(precision=2)

In [6]:
arithmetic_operations = [
    ((lambda x, y: x + y), "+"),
    ((lambda x, y: x - y), "-"),
    ((lambda x, y: x * y), "*"),
    ((lambda x, y: x / y), "/"),
    ((lambda x, y: x // y), "//"),
]

bitwise_operations = [
    ((lambda x, y: x | y), "|"),
    ((lambda x, y: x & y), "&"),
    ((lambda x, y: x ^ y), "^"),
]

np_types = [
    np.int8,
    np.int16,
    np.int32,
    np.int64,
    np.uint8,
    np.uint16,
    np.uint32,
    np.uint64,
    np.float32,
    np.float64,
]

for op, op_name in arithmetic_operations:
    for left_type in [np.uint8]:
        for right_type in [np.uint8, np.int32, np.float32]:
            pipe = arithmetic_pipeline(op, left_type, right_type)
            build_and_run(pipe, op_name)

for op, op_name in bitwise_operations:
    for left_type in [np.uint8]:
        for right_type in [np.uint8, np.int32]:
            pipe = arithmetic_pipeline(op, left_type, right_type)
            build_and_run(pipe, op_name)

[[42  8]] + [[9 2]] = [[51 10]]; 
	with types uint8 + uint8 -> uint8

[[42  8]] + [[9 2]] = [[51 10]]; 
	with types uint8 + int32 -> int32

[[42  8]] + [[9. 2.]] = [[51. 10.]]; 
	with types uint8 + float32 -> float32

[[42  8]] - [[9 2]] = [[33  6]]; 
	with types uint8 - uint8 -> uint8

[[42  8]] - [[9 2]] = [[33  6]]; 
	with types uint8 - int32 -> int32

[[42  8]] - [[9. 2.]] = [[33.  6.]]; 
	with types uint8 - float32 -> float32

[[42  8]] * [[9 2]] = [[122  16]]; 
	with types uint8 * uint8 -> uint8

[[42  8]] * [[9 2]] = [[378  16]]; 
	with types uint8 * int32 -> int32

[[42  8]] * [[9. 2.]] = [[378.  16.]]; 
	with types uint8 * float32 -> float32

[[42  8]] / [[9 2]] = [[4.67 4.  ]]; 
	with types uint8 / uint8 -> float32

[[42  8]] / [[9 2]] = [[4.67 4.  ]]; 
	with types uint8 / int32 -> float32

[[42  8]] / [[9. 2.]] = [[4.67 4.  ]]; 
	with types uint8 / float32 -> float32

[[42  8]] // [[9 2]] = [[4 4]]; 
	with types uint8 // uint8 -> uint8

[[42  8]] // [[9 2]] = [[4 4]]; 
	with

## Using Constants

Instead of operating only on Tensor data, DALI expressions can also work with constants. The constants can be values of the Python int and float types that are used directly, or those values are wrapped in `nvidia.dali.types.Constant`. The operation on the tensor and constant results in the constant that is being broadcasted to all tensor elements.  

**Note**: The same constant is used with all samples in the batch. Currently, the values of integral constants are passed to DALI as `int32`, and the values of the floating point constants are passed to DALI as `float32`. 

Regarding the promotions type, the Python int values will be treated as `int32`, and the float as `float32`. 

The DALI `Constant` can be used to indicate other types. It accepts `DALIDataType` enum values as second argument and has convenience member functions like `.uint8()` or `.float32()` that can be used for conversions.

1. The expressions in this examples consist of a tensor and a constant, so you can adjust your previous pipeline and the helper functions. These functions need to generate only one tensor.

In [7]:
def arithmetic_constant_pipeline(operation, tensor_data_type):
    pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
    with pipe:
        t = fn.external_source(
            source=lambda: get_data(tensor_data_type, tensor_data_type)[0],
            dtype=dali_type(tensor_data_type),
        )
        pipe.set_outputs(t, operation(t))

    return pipe


def build_and_run_with_const(pipe, op_name, constant, is_const_left=False):
    pipe.build()
    pipe_out = pipe.run()
    t_in = pipe_out[0].as_array()
    t_out = pipe_out[1].as_array()
    if is_const_left:
        print(
            "{} {} {} = \n{}; \n\twith types {} {} {} -> {}\n".format(
                constant,
                op_name,
                t_in,
                t_out,
                type(constant),
                op_name,
                t_in.dtype,
                t_out.dtype,
            )
        )
    else:
        print(
            "{} {} {} = \n{}; \n\twith types {} {} {} -> {}\n".format(
                t_in,
                op_name,
                constant,
                t_out,
                t_in.dtype,
                op_name,
                type(constant),
                t_out.dtype,
            )
        )

The `ArithmeticConstantsPipeline` can be parametrized with a function that takes the only tensor and returns the result of arithmetic operation between that tensor and a constant. The print message was also adjusted.

2. Check the examples that were mentioned at the beginning:
  - `int`
  - `float`
  - `nvidia.dali.types.Constant`.

In [8]:
constant = 10
pipe = arithmetic_constant_pipeline((lambda x: x + constant), np.uint8)
build_and_run_with_const(pipe, "+", constant)

constant = 10
pipe = arithmetic_constant_pipeline((lambda x: x + constant), np.float32)
build_and_run_with_const(pipe, "+", constant)


constant = 42.3
pipe = arithmetic_constant_pipeline((lambda x: x + constant), np.uint8)
build_and_run_with_const(pipe, "+", constant)

constant = 42.3
pipe = arithmetic_constant_pipeline((lambda x: x + constant), np.float32)
build_and_run_with_const(pipe, "+", constant)

[[42  8]] + 10 = 
[[52 18]]; 
	with types uint8 + <class 'int'> -> int32

[[42.  8.]] + 10 = 
[[52. 18.]]; 
	with types float32 + <class 'int'> -> float32

[[42  8]] + 42.3 = 
[[84.3 50.3]]; 
	with types uint8 + <class 'float'> -> float32

[[42.  8.]] + 42.3 = 
[[84.3 50.3]]; 
	with types float32 + <class 'float'> -> float32



The value of the constant is applied to all the elements of the tensor to which it is added.

3. Check how to use the DALI Constant wrapper.

4. Passing an `int` or `float` to a DALI Constant marks it as `int32` or `float32` respectively

In [9]:
constant = Constant(10)
pipe = arithmetic_constant_pipeline((lambda x: x * constant), np.uint8)
build_and_run_with_const(pipe, "*", constant)


constant = Constant(10.0)
pipe = arithmetic_constant_pipeline((lambda x: constant * x), np.uint8)
build_and_run_with_const(pipe, "*", constant, True)

[[42  8]] * 10:DALIDataType.INT32 = 
[[420  80]]; 
	with types uint8 * <class 'nvidia.dali.types.ScalarConstant'> -> int32

10.0:DALIDataType.FLOAT * [[42  8]] = 
[[420.  80.]]; 
	with types <class 'nvidia.dali.types.ScalarConstant'> * uint8 -> float32



5. Explicitly specify the type as a second argument, or use convenience conversion member functions.

In [10]:
constant = Constant(10, types.DALIDataType.UINT8)
pipe = arithmetic_constant_pipeline((lambda x: x * constant), np.uint8)
build_and_run_with_const(pipe, "*", constant)


constant = Constant(10.0, types.DALIDataType.UINT8)
pipe = arithmetic_constant_pipeline((lambda x: constant * x), np.uint8)
build_and_run_with_const(pipe, "*", constant, True)


constant = Constant(10).uint8()
pipe = arithmetic_constant_pipeline((lambda x: constant * x), np.uint8)
build_and_run_with_const(pipe, "*", constant, True)

[[42  8]] * 10:DALIDataType.UINT8 = 
[[164  80]]; 
	with types uint8 * <class 'nvidia.dali.types.ScalarConstant'> -> uint8

10:DALIDataType.UINT8 * [[42  8]] = 
[[164  80]]; 
	with types <class 'nvidia.dali.types.ScalarConstant'> * uint8 -> uint8

10:DALIDataType.UINT8 * [[42  8]] = 
[[164  80]]; 
	with types <class 'nvidia.dali.types.ScalarConstant'> * uint8 -> uint8



## Treating Tensors as Scalars

If one of the tensors is considered a scalar input, the same rules apply.
